In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#Pyramid
def batch_pyramid(images):
    pyramid_list = []
    for i in tqdm(range(len(images))):
        img = (images.astype(np.float32)* 255)[i, :, :, :].astype(np.uint8)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        G = img.copy()
        gp = [G]
        for i in range(4):
            G = cv2.pyrDown(G)
            gp.append(G)
        gp_out = [cv2.resize(gp[2], (128,128)),cv2.resize(gp[3], (128,128))]
        pyramid_list.append((np.array(gp_out)).astype(np.uint8))
    pyramid_list = np.array(pyramid_list,dtype = float32).reshape([images.shape[0],images.shape[1],images.shape[2],2])
    
    print (pyramid_list.shape)
    return pyramid_list

In [3]:
# def atoi(text):
#     return int(text) if text.isdigit() else text

# def natural_keys(text):
#     return [ atoi(c) for c in re.split('(\d+)', text) ]

# def load_flower_data():
#     # grab the list of images that we'll be describing
#     print("[INFO] handling images...")
#     TRAIN_ORIGINAL_DIR = '../train/'
#     TRAIN_SUB_DIR = '../subsample/'
#     TRAIN_GAN = '../../image_gan/'
#     TEST_DIR = '../../test/'

#     # use this for full dataset
#     train_images_gan = [TRAIN_GAN + i for i in os.listdir(TRAIN_GAN)]
#     test_images = [TEST_DIR + i for i in os.listdir(TEST_DIR)]
    
#     train_images = train_images_gan
    
#     train_images.sort(key=natural_keys)
#     test_images.sort(key=natural_keys)

#     # initialize the features matrix and labels list
#     trainImage = []
#     trainLabels = []
#     testImage = []
#     testLabels = []

#     # loop over the input images
#     for (i, imagePath) in enumerate(train_images):
#         # extract the class label
#         # get the labels from the name of the images by extract the string before "_"
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # read and resize image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         trainImage.append(img)
#         trainLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(train_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(train_images)))
            
#       # loop over the input images
#     for (i, imagePath) in enumerate(test_images):
#         # extract the class label
#         # our images were named as labels.image_number.format
#         # get the labels from the name of the images by extract the string before "."
#         label = imagePath.split(os.path.sep)[-1].split("_")[0]

#         # extract CNN features in the image
#         img = cv2.imread(imagePath)
#         img = cv2.resize(img, (128,128))

#         # add the messages we got to features and labels matricies
#         testImage.append(img)
#         testLabels.append(label)

#         # show an update every 100 images until the last image
#         if i > 0 and ((i + 1) % 1000 == 0 or i == len(test_images) - 1):
#             print("[INFO] processed {}/{}".format(i + 1, len(test_images)))


#     trainImage = np.array(trainImage,dtype = float32)
#     trainLabels = np.array(trainLabels)
#     testImage = np.array(testImage,dtype = float32)
#     testLabels = np.array(testLabels)
#     print (trainImage.shape)
    
#     trainImage = trainImage.astype(np.float32) / 255
#     testImage = testImage.astype(np.float32) / 255
    
#     le = preprocessing.LabelEncoder()
#     le.fit(trainLabels)
#     list(le.classes_)
#     trainLabels = le.transform(trainLabels) 
#     testLabels = le.transform(testLabels) 
    
#     return trainImage, trainLabels, testImage, testLabels

In [4]:
# trainImage, trainLabels, testImage, testLabels = load_flower_data()

# trainImage_pyramid = batch_pyramid(trainImage)
# testImage_pyramid = batch_pyramid(testImage)
# nb_classes = 2

# # Convert class vectors to binary class matrices.
# trainLabels = keras.utils.to_categorical(trainLabels, nb_classes)
# print (trainLabels)
# testLabels = keras.utils.to_categorical(testLabels, nb_classes)
# print (testLabels)
# print (testLabels.shape)

# np.save('../trainImage.npy', trainImage)
# np.save('../trainLabels.npy', trainLabels)
# np.save('../testImage.npy', testImage)
# np.save('../testLabels.npy', testLabels)
# np.save('../trainImage_pyramid_G.npy', trainImage_pyramid)
# np.save('../testImage_pyramid_G.npy', testImage_pyramid)

# print("[INFO] trainImage matrix: {:.2f}MB".format(
#     (trainImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainLabels matrix: {:.4f}MB".format(
#     (trainLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage matrix: {:.2f}MB".format(
#     (testImage.nbytes) / (1024 * 1000.0)))
# print("[INFO] testLabels matrix: {:.4f}MB".format(
#     (testLabels.nbytes) / (1024 * 1000.0)))
# print("[INFO] trainImage_pyramid matrix: {:.2f}MB".format(
#     (trainImage_pyramid.nbytes) / (1024 * 1000.0)))
# print("[INFO] testImage_pyramid matrix: {:.4f}MB".format(
#     (testImage_pyramid.nbytes) / (1024 * 1000.0)))


[INFO] handling images...
[INFO] processed 1000/2000
[INFO] processed 2000/2000
[INFO] processed 158/158
(2000, 128, 128, 3)


  8%|▊         | 13/158 [00:00<00:01, 124.63it/s]

(2000, 128, 128, 2)


100%|██████████| 158/158 [00:01<00:00, 128.63it/s]


(158, 128, 128, 2)
[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [0. 1.]
 [0. 1.]]
[[1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0

In [5]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size
        
        self.classname = ['Iris', 'Pansy']

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.pyramid_dim = 2  # color dimension
        self.nb_class = 2
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Pyramid_G_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_pyramid = np.load('../trainImage_pyramid_L.npy')
        self.test_x_pyramid = np.load('../testImage_pyramid_L.npy')
        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_pyramid, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_pyramid",x_pyramid.get_shape()) # 128, 128, 3 
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_pyramid,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_pyramid = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.pyramid_dim], 
                                name='x_pyramid')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x, self.x_pyramid, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_pyramid = self.train_x_pyramid[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_pyramid= shuffled_set_pyramid[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_pyramid: batch_x_pyramid,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_pyramid: batch_x_pyramid_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramid_test =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_pyramidtest =self.test_x_pyramid[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_pyramid: batch_x_pyramid_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
dataset = '4_Flowers_1s'
epoch = 50
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
#  [*] Best Epoch:  28 , Accuracy:  0.7999999523162842
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Pyramid_G_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Pyramid_G_C5_D1_Kernel(3,3)_128_lrdecay-28
#  [*] Finished testing Best Epoch: 28 , accuracy:  0.7999999523162842 !

# Avg per epoch ptime: 5.10, total 50 epochs ptime: 273.10
#  [*] Training finished!
#  [*] Best Epoch:  18 , Accuracy:  0.8500000238418579
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Pyramid_G_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Pyramid_G_C5_D1_Kernel(3,3)_128_lrdecay-18
#  [*] Finished testing Best Epoch: 18 , accuracy:  0.8500000238418579 !

CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_pyramid (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 64)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 2)
------------------------
---------
Variables: name (type shape) [size]
---------
cnn/conv_1_1/kernel:0 (float32_ref 3x3x3x32) [864, bytes: 3456]
cnn/conv_1_1/bias:0 (float32_ref 32) [32, bytes: 128]
cnn/conv_1_2/kernel:0 (float32_ref 3x3x2x32) [576, bytes: 2304]
cnn/co

Epoch: [ 5] [  16/  20] time: 27.2216, loss: 0.00396382
Epoch: [ 5] [  17/  20] time: 27.4615, loss: 0.00971569
Epoch: [ 5] [  18/  20] time: 27.6992, loss: 0.01792888
Epoch: [ 5] [  19/  20] time: 27.9380, loss: 0.02458555
[5/50] - ptime: 5.0362 loss: 0.03134133 acc: 0.78000 lr: 0.00100000
Epoch: [ 6] [   0/  20] time: 28.8479, loss: 0.00889613
Epoch: [ 6] [   1/  20] time: 29.0844, loss: 0.02998620
Epoch: [ 6] [   2/  20] time: 29.3245, loss: 0.02045661
Epoch: [ 6] [   3/  20] time: 29.5630, loss: 0.02132231
Epoch: [ 6] [   4/  20] time: 29.8015, loss: 0.04078565
Epoch: [ 6] [   5/  20] time: 30.0407, loss: 0.00900039
Epoch: [ 6] [   6/  20] time: 30.2818, loss: 0.02459550
Epoch: [ 6] [   7/  20] time: 30.5225, loss: 0.01126353
Epoch: [ 6] [   8/  20] time: 30.7611, loss: 0.02309922
Epoch: [ 6] [   9/  20] time: 30.9989, loss: 0.01149140
Epoch: [ 6] [  10/  20] time: 31.2383, loss: 0.01130339
Epoch: [ 6] [  11/  20] time: 31.4773, loss: 0.03808407
Epoch: [ 6] [  12/  20] time: 31.714

Epoch: [12] [  14/  20] time: 64.6621, loss: 0.00251351
Epoch: [12] [  15/  20] time: 64.9004, loss: 0.00013421
Epoch: [12] [  16/  20] time: 65.1387, loss: 0.00060437
Epoch: [12] [  17/  20] time: 65.3783, loss: 0.00040199
Epoch: [12] [  18/  20] time: 65.6162, loss: 0.00309230
Epoch: [12] [  19/  20] time: 65.8560, loss: 0.00009235
[12/50] - ptime: 5.0535 loss: 0.00240983 acc: 0.82000 lr: 0.00090000
Epoch: [13] [   0/  20] time: 66.7197, loss: 0.00096265
Epoch: [13] [   1/  20] time: 66.9574, loss: 0.00137912
Epoch: [13] [   2/  20] time: 67.2000, loss: 0.00677651
Epoch: [13] [   3/  20] time: 67.4410, loss: 0.00105344
Epoch: [13] [   4/  20] time: 67.6793, loss: 0.00223841
Epoch: [13] [   5/  20] time: 67.9176, loss: 0.00192875
Epoch: [13] [   6/  20] time: 68.1577, loss: 0.02147016
Epoch: [13] [   7/  20] time: 68.4005, loss: 0.00015801
Epoch: [13] [   8/  20] time: 68.6400, loss: 0.01695956
Epoch: [13] [   9/  20] time: 68.8796, loss: 0.00355072
Epoch: [13] [  10/  20] time: 69.11

Epoch: [19] [  12/  20] time: 102.1859, loss: 0.00006191
Epoch: [19] [  13/  20] time: 102.4278, loss: 0.00246348
Epoch: [19] [  14/  20] time: 102.6677, loss: 0.00085639
Epoch: [19] [  15/  20] time: 102.9082, loss: 0.00171179
Epoch: [19] [  16/  20] time: 103.1491, loss: 0.00003291
Epoch: [19] [  17/  20] time: 103.3916, loss: 0.00005389
Epoch: [19] [  18/  20] time: 103.6305, loss: 0.00015532
Epoch: [19] [  19/  20] time: 103.8702, loss: 0.00151418
[19/50] - ptime: 5.0680 loss: 0.00144703 acc: 0.80000 lr: 0.00090000
Epoch: [20] [   0/  20] time: 104.7317, loss: 0.00013203
Epoch: [20] [   1/  20] time: 104.9702, loss: 0.00015456
Epoch: [20] [   2/  20] time: 105.2113, loss: 0.00069818
Epoch: [20] [   3/  20] time: 105.4533, loss: 0.00013117
Epoch: [20] [   4/  20] time: 105.6938, loss: 0.00034899
Epoch: [20] [   5/  20] time: 105.9330, loss: 0.00127127
Epoch: [20] [   6/  20] time: 106.1729, loss: 0.00125610
Epoch: [20] [   7/  20] time: 106.4201, loss: 0.00003673
Epoch: [20] [   8/ 

Epoch: [26] [   8/  20] time: 139.3385, loss: 0.00007413
Epoch: [26] [   9/  20] time: 139.5784, loss: 0.00011771
Epoch: [26] [  10/  20] time: 139.8180, loss: 0.00075853
Epoch: [26] [  11/  20] time: 140.0579, loss: 0.00003245
Epoch: [26] [  12/  20] time: 140.3018, loss: 0.00011733
Epoch: [26] [  13/  20] time: 140.5426, loss: 0.00004312
Epoch: [26] [  14/  20] time: 140.7828, loss: 0.00003279
Epoch: [26] [  15/  20] time: 141.0232, loss: 0.00079984
Epoch: [26] [  16/  20] time: 141.2637, loss: 0.00002087
Epoch: [26] [  17/  20] time: 141.5050, loss: 0.00018427
Epoch: [26] [  18/  20] time: 141.7475, loss: 0.00001970
Epoch: [26] [  19/  20] time: 141.9866, loss: 0.00016181
[26/50] - ptime: 5.0826 loss: 0.00020318 acc: 0.73000 lr: 0.00081000
Epoch: [27] [   0/  20] time: 142.8473, loss: 0.00020062
Epoch: [27] [   1/  20] time: 143.0824, loss: 0.00000517
Epoch: [27] [   2/  20] time: 143.3214, loss: 0.00005127
Epoch: [27] [   3/  20] time: 143.5630, loss: 0.00002669
Epoch: [27] [   4/ 

Epoch: [33] [   4/  20] time: 176.4846, loss: 0.00008807
Epoch: [33] [   5/  20] time: 176.7239, loss: 0.00004590
Epoch: [33] [   6/  20] time: 176.9643, loss: 0.00001291
Epoch: [33] [   7/  20] time: 177.2045, loss: 0.00000338
Epoch: [33] [   8/  20] time: 177.4483, loss: 0.00001389
Epoch: [33] [   9/  20] time: 177.6855, loss: 0.00000424
Epoch: [33] [  10/  20] time: 177.9271, loss: 0.00001770
Epoch: [33] [  11/  20] time: 178.1679, loss: 0.00000416
Epoch: [33] [  12/  20] time: 178.4122, loss: 0.00001167
Epoch: [33] [  13/  20] time: 178.6522, loss: 0.00001955
Epoch: [33] [  14/  20] time: 178.8922, loss: 0.00002605
Epoch: [33] [  15/  20] time: 179.1326, loss: 0.00001127
Epoch: [33] [  16/  20] time: 179.3731, loss: 0.00007371
Epoch: [33] [  17/  20] time: 179.6125, loss: 0.00001171
Epoch: [33] [  18/  20] time: 179.8515, loss: 0.00003491
Epoch: [33] [  19/  20] time: 180.0905, loss: 0.00004983
[33/50] - ptime: 5.0810 loss: 0.00003622 acc: 0.70000 lr: 0.00072900
Epoch: [34] [   0/ 

Epoch: [40] [   0/  20] time: 213.6075, loss: 0.00004192
Epoch: [40] [   1/  20] time: 213.8458, loss: 0.00001009
Epoch: [40] [   2/  20] time: 214.0866, loss: 0.00000201
Epoch: [40] [   3/  20] time: 214.3270, loss: 0.00000065
Epoch: [40] [   4/  20] time: 214.5689, loss: 0.00000401
Epoch: [40] [   5/  20] time: 214.8089, loss: 0.00003225
Epoch: [40] [   6/  20] time: 215.0497, loss: 0.00000431
Epoch: [40] [   7/  20] time: 215.2895, loss: 0.00001596
Epoch: [40] [   8/  20] time: 215.5326, loss: 0.00000974
Epoch: [40] [   9/  20] time: 215.7723, loss: 0.00000683
Epoch: [40] [  10/  20] time: 216.0130, loss: 0.00004581
Epoch: [40] [  11/  20] time: 216.2528, loss: 0.00000914
Epoch: [40] [  12/  20] time: 216.4960, loss: 0.00000510
Epoch: [40] [  13/  20] time: 216.7371, loss: 0.00000441
Epoch: [40] [  14/  20] time: 216.9777, loss: 0.00001122
Epoch: [40] [  15/  20] time: 217.2190, loss: 0.00004104
Epoch: [40] [  16/  20] time: 217.4558, loss: 0.00001245
Epoch: [40] [  17/  20] time: 2

Epoch: [46] [  17/  20] time: 250.3642, loss: 0.00000649
Epoch: [46] [  18/  20] time: 250.6043, loss: 0.00000456
Epoch: [46] [  19/  20] time: 250.8443, loss: 0.00001376
[46/50] - ptime: 5.0945 loss: 0.00001040 acc: 0.67000 lr: 0.00065610
Epoch: [47] [   0/  20] time: 251.7024, loss: 0.00002575
Epoch: [47] [   1/  20] time: 251.9400, loss: 0.00001336
Epoch: [47] [   2/  20] time: 252.1797, loss: 0.00000260
Epoch: [47] [   3/  20] time: 252.4235, loss: 0.00001338
Epoch: [47] [   4/  20] time: 252.6647, loss: 0.00006500
Epoch: [47] [   5/  20] time: 252.9048, loss: 0.00002754
Epoch: [47] [   6/  20] time: 253.1449, loss: 0.00001016
Epoch: [47] [   7/  20] time: 253.3863, loss: 0.00016731
Epoch: [47] [   8/  20] time: 253.6266, loss: 0.00004619
Epoch: [47] [   9/  20] time: 253.8676, loss: 0.00004071
Epoch: [47] [  10/  20] time: 254.1072, loss: 0.00000938
Epoch: [47] [  11/  20] time: 254.3479, loss: 0.00000397
Epoch: [47] [  12/  20] time: 254.5885, loss: 0.00000328
Epoch: [47] [  13/ 

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放